In [1]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from cont_blobs import *
import lsst.sims.featureScheduler.detailers as detailers
from lsst.sims.featureScheduler.utils import  schema_converter

In [2]:

nside = 32
per_night = True  # Dither DDF per night
nexp = 1  # All observations
mixed_pairs = True  # For the blob scheduler
camera_ddf_rot_limit = 75.

max_dither = 0.7

dither_detailer = detailers.Dither_detailer(per_night=per_night, max_dither=max_dither)
details = [detailers.Camera_rot_detailer(min_rot=-camera_ddf_rot_limit, max_rot=camera_ddf_rot_limit), dither_detailer]
ddfs = generate_dd_surveys(nside=nside, nexp=nexp, detailers=details)

greedy = gen_greedy_surveys(nside, nexp=nexp)
blobs = generate_blobs(nside, nexp=nexp)
surveys = [ddfs, blobs, greedy]

In [3]:
scheduler = Core_scheduler(surveys, nside=nside)
observatory = Model_observatory(nside=nside)



In [4]:
sco = schema_converter()
observations = sco.opsim2obs('cont_blobs_v1.4_10yrs.db')

In [17]:
np.where(observations['alt'] < np.radians(10))[0]

array([ 841888,  841918, 1018989, 1019023, 1543808, 1543838, 1711904,
       1711938, 1712536, 1712559])

In [23]:
observations['note'][1018989], observations['RA'][1019023]

('blob, yy, a', 1.2393238483133215)

In [6]:
# 
i = 841888-10
for obs in observations[0:i]:
    scheduler.add_observation(obs)

In [8]:
# Set the observatory to the correct state. Takes a while because it has to load the sky brightness file.
observatory.mjd = obs['mjd']
# Observatory starts parked, so need to send an expose command to slew to the correct position
temp = observatory.observe(obs)
# It took some time to make that slew, so reset the time again
observatory.mjd = obs['mjd']
should_match_obs, new_night = observatory.observe(obs)
should_match_obs == obs